In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

# Install and load all libraries

In [2]:
from utils import *

# Creating differe dataloaders

In [3]:
learn = load_learner('models/06_archs-small-squish-convnext_large_in22k.pkl',cpu=False)
learn.model.eval();

In [ ]:
x, = first(dls.test_dl(['data/train_images_tiny/clip_000003/00000.jpg'])); 
x.shape

In [ ]:
cls,idx,prob=learn.predict('data/train_images_tiny/clip_000003/00000.jpg')

In [5]:
learn.predict??

Signature: learn.predict(item, rm_type_tfms=None, with_input=False)
Docstring: Prediction on `item`, fully decoded, loss function decoded and probabilities
Source:   
    def predict(self, item, rm_type_tfms=None, with_input=False):
        dl = self.dls.test_dl([item], rm_type_tfms=rm_type_tfms, num_workers=0)
        inp,preds,_,dec_preds = self.get_preds(dl=dl, with_input=True, with_decoded=True)
        i = getattr(self.dls, 'n_inp', -1)
        inp = (inp,) if i==1 else tuplify(inp)
        dec = self.dls.decode_batch(inp + tuplify(dec_preds))[0]
        dec_inp,dec_targ = map(detuplify, [dec[:i],dec[i:]])
        res = dec_targ,dec_preds[0],preds[0]
        if with_input: res = (dec_inp,) + res
        return res
File:      ~/mambaforge/lib/python3.9/site-packages/fastai/learner.py
Type:      method


In [6]:
 inp,preds,_,dec_preds=learn.get_preds(dl=dls.test_dl(['data/train_images_tiny/clip_000003/00000.jpg']), with_input=True, with_decoded=True)

In [9]:
inp.shape, preds.shape, dec_preds

(torch.Size([1, 3, 180, 320]),
 torch.Size([1, 60]),
 TensorBase([[-8.6797, -1.5264, -0.3330, -6.8320, -6.2188,  0.8818, -7.1992, -8.7422,
           5.5938, -8.6953, -7.1992, -8.8047, -8.6875, -6.4766, -7.1953, -8.6797,
           1.3633, -3.9277, -4.8242, -7.1055, -8.6328, -4.1562,  6.6523, -3.9180,
          -4.7383, -8.1484, -8.5625, -8.5625, -8.7031, -4.8984, -8.7656, -6.8086,
          -7.2188, -6.1562, -8.7734, -8.8125, -1.7637,  0.4358,  5.9766, -7.1133,
          -8.7812, -4.9102, -6.3828, -8.8984, -6.4727, -7.1992, -4.5195,  4.0430,
          -2.0957, -5.5508,  1.4893, -2.4102, -8.2109, -3.4258, -6.2852, -2.9375,
          -5.6445, -6.0156, -4.8203, -1.5850]]))

In [27]:
tuplify(dec_preds[:,:15].softmax(1).argmax().item())

(8,)

In [28]:
learn.dls.decode_batch(inp + tuplify(dec_preds[:,:15].softmax(1).argmax()))[0]

TypeError: unsupported operand type(s) for +: 'TensorImage' and 'tuple'

In [ ]:
%debug

In [ ]:
learn.dls.decode_batch??

In [ ]:
learn.dls._decode_batch??

In [ ]:
output = learn.model.eval()(x)

In [ ]:
learn.dls.vocab[0][output[:,:cfg(1)].softmax(1).argmax(1).item()]

In [ ]:
doc(Datasets.decode)

In [ ]:
ifnone??

In [ ]:
tuplify(listify([0,1,2,3],[2,3,4,5],[2,2,3]))[0]

In [ ]:
class Hook():
    def hook_func(self, m, i, o): self.stored = o.detach().clone()

In [ ]:
hook_output = Hook()
hook = learn.model[0].register_forward_hook(hook_output.hook_func)

In [ ]:
with torch.no_grad(): output = learn.model.eval()(x)

In [ ]:
act = hook_output.stored[0]

In [ ]:
F.softmax(output[:,:cfg(1)], dim=-1)

In [ ]:
learn.model[1][-1].weight.shape, act.shape

In [ ]:
cam_map = torch.einsum('ck,kij->cij', learn.model[1][-1].weight, act)
cam_map.shape

In [ ]:
class Hook():
    def __init__(self, m):
        self.hook = m.register_forward_hook(self.hook_func)   
    def hook_func(self, m, i, o): self.stored = o.detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()

In [ ]:
with Hook(learn.model[0]) as hook:
    with torch.no_grad(): output = learn.model.eval()(x.cuda())
    act = hook.stored

In [ ]:
class HookBwd():
    def __init__(self, m):
        self.hook = m.register_backward_hook(self.hook_func)   
    def hook_func(self, m, gi, go): self.stored = go[0].detach().clone()
    def __enter__(self, *args): return self
    def __exit__(self, *args): self.hook.remove()

In [ ]:
cls = 1
with HookBwd(learn.model[0]) as hookg:
    with Hook(learn.model[0]) as hook:
        output = learn.model.eval()(x.cuda())
        act = hook.stored
    output[0,cls].backward()
    grad = hookg.stored

In [ ]:
w = grad[0].mean(dim=[1,2], keepdim=True)
cam_map = (w * act[0]).sum(0)

In [ ]:
_,ax = plt.subplots()
x.show(ctx=ax)
ax.imshow(cam_map.detach().cpu(), alpha=0.6, extent=(1, 3, 180, 320),
              interpolation='bilinear', cmap='magma');

In [ ]:
def train(exp_name, arch, size, item_tfms, accum=1, finetune=True, epochs=12):
    dblock = get_dblock(item_tfms=item_tfms, batch_tfms=aug_transforms(size=size, min_scale=1))
    dls = dblock.dataloaders(images_df, seed=42, batch_size=32, n_workers=8)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, loss_func=combo_loss, metrics=metrics_cfg, n_out=cfg(4), cbs=cbs).to_fp16()
    if finetune:
        learn.fine_tune(epochs, 0.01)
        learn.path=Path('models')
        learn.export(f'06_archs-small-{exp_name}-{arch}.pkl')
        return learn.tta(dl=dls.valid)
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, 0.01)


In [ ]:
res = (180,320)

In [ ]:
archs = {
    'convnext_large_in22k': {
        (Resize(res, ResizeMethod.Squish), (180,320)),
    },
    'vit_large_patch16_224': {
        (Resize(res, ResizeMethod.Squish), 224),
    },
    'swinv2_large_window12_192_22k':{
        (Resize(res, ResizeMethod.Squish), 192),
    },
    'swin_large_patch4_window7_224': {
        (Resize(res, ResizeMethod.Squish), 224)
    }
}

In [ ]:
tta_res = []

for arch, details in archs.items():
    for i, s in details:
        print('----',arch, ' --- ')
        print(s)
        print(i.name)
        tta_res.append(train('squish', arch, size=s, item_tfms=i))
        gc.collect()
        torch.cuda.empty_cache()


In [ ]:
save_pickle('07_archs-large-ensemble-tta_res.pkl', tta_res)

In [ ]:
def decode_ensemble_result(output):
    output = first(zip(*output)) # unzipping preds and targes
    output+= output[1:3]        # increase weights of certain models with higher accuracy
    output = torch.stack(output)# stacking predctions from all models
    labels=[]
    df = pd.DataFrame(dls.valid.items)
    df.drop(['y1_label','y2_label','y3_label','y4_label','valid'], axis=1, inplace=True)
    for i in range(len(dls.c)):
        arm_preds = output[:,:,cfg(i):cfg(i+1)].mean(0);
        arm_idxs = arm_preds.argmax(dim=1)
        arm_vocab = np.array(dls.vocab[i])
        df[f'y{i+1}_label'] = arm_vocab[arm_idxs]
        
    df['labels_pred'] = '['+ df['y1_label']+', '+df['y2_label']+', '+df['y3_label']+', '+df['y4_label']+']'
    df['accurate_tf'] = (df['labels']==df['labels_pred'])
    df.drop(['y1_label','y2_label','y3_label','y4_label'], axis=1, inplace=True)
    return df

In [ ]:
output_df = decode_ensemble_result(tta_res)
output_df.to_csv('07_archs-larger-ensemble-results.csv', index=False)
output_df['accurate_tf'].value_counts()/len(output_df)*100

In [ ]:
tta_prs = first(zip(*tta_res))

In [ ]:
tta_prs+= tta_prs[1:3]

In [ ]:
avg_pr = torch.stack(tta_prs).mean(0)

In [ ]:
tta_prs1 = tta_prs[:][:,:cfg(1)]

In [ ]:
tta_prs[0][:,cfg(1):cfg(2)].shape

In [ ]:
dblock.dataloaders??

In [ ]:
def decode_preds (preds):
    for i,p in enumerate(preds):
        lbl = []
        for i in range(len(dls.c)):
            v = {k:v for k,v in enumerate(dls.vocab[i])}
            idx = p[cfg(i):cfg(i+1)].softmax(0).argmax().item()
            lbl.append(v[idx])
        print(lbl)

In [ ]:
decode_preds(preds)

In [ ]:
df = pd.read_csv('images_df.csv')
df = df.query("valid==True").reset_index(drop=False)
df.drop(columns=['index'], axis=1)
df.head()

In [ ]:
img = PILImage.create('data/train_images_tiny/clip_000003/00000.jpg')
img.shape

In [ ]:
x, = first(dls.test_dl(['data/train_images_tiny/clip_000003/00000.jpg'])); 
x.shape

In [ ]:
inp,preds,_,dec_preds = learn.get_preds(dl=dls.test_dl('data/train_images_tiny/clip_000003/00000.jpg'), with_input=True, with_decoded=True)

In [ ]:
type(inp)

In [ ]:
outs = tuplify([preds[:,:cfg(1)], preds[:,cfg(1):cfg(2)], preds[:,cfg(2):cfg(3)],preds[:,cfg(3):cfg(4)]])

In [ ]:
for i in range(len(outs)):
    print(outs[i])

In [ ]:
list(range(len(outs)))